In [ ]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame, permitiendo la detección de tipo más precisa
df = pd.read_csv('generadoras_activas_2020-2023', low_memory=False)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.clustering import TimeSeriesKMeans
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Configurar el dispositivo para usar CUDA si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando el dispositivo: {device}")

# Cargar el archivo CSV
df = pd.read_csv('generadoras_activas_corregido.csv', low_memory=False)
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Agrupar y preparar los datos
energia_por_fecha = df.groupby(['LLAVE NOMBRE', 'FECHA'])['TOTAL'].sum().reset_index()
pivot_df = energia_por_fecha.pivot(index='LLAVE NOMBRE', columns='FECHA', values='TOTAL').fillna(0)
pivot_df.columns = pd.to_datetime(pivot_df.columns)
pivot_df['LLAVE NOMBRE'] = pivot_df.index

# Separar en 90% entrenamiento y 10% validación
train_df, val_df = train_test_split(pivot_df, test_size=0.1, random_state=42)
train_data = train_df.drop(columns=['LLAVE NOMBRE']).values
val_data = val_df.drop(columns=['LLAVE NOMBRE']).values

# Normalización de series temporales
scaler = TimeSeriesScalerMinMax()
train_data_normalized = scaler.fit_transform(train_data.reshape((train_data.shape[0], train_data.shape[1], 1)))
val_data_normalized = scaler.transform(val_data.reshape((val_data.shape[0], val_data.shape[1], 1)))

# Convertir los datos a tensores y cargar al dispositivo
train_tensor = torch.Tensor(train_data_normalized).to(device)
train_dataset = TensorDataset(train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Definir el modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1, dropout_rate=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = True
        self.num_directions = 2 if self.bidirectional else 1
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            bidirectional=self.bidirectional, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(hidden_size * self.num_directions, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h_0 = torch.randn(self.num_directions * self.num_layers, batch_size, self.hidden_size).to(x.device)
        c_0 = torch.randn(self.num_directions * self.num_layers, batch_size, self.hidden_size).to(x.device)
        lstm_out, _ = self.lstm(x, (h_0, c_0))
        out = self.dropout(lstm_out[:, -1, :])
        out = self.linear(out)
        return out

# Exploración de hiperparámetros del LSTM
hidden_sizes = [32, 64, 128]
num_layers_list = [1, 2, 3]
learning_rates = [0.001, 0.0001]
dropout_rates = [0.2, 0.3]

best_model = None
best_loss = float('inf')
best_params = {}
results = []

# Medir el tiempo de inicio de la búsqueda de hiperparámetros
start_time = time.time()

# Probar diferentes combinaciones de hiperparámetros
for hidden_size in hidden_sizes:
    for num_layers in num_layers_list:
        for lr in learning_rates:
            for dropout_rate in dropout_rates:
                model = LSTMModel(input_size=1, hidden_size=hidden_size, num_layers=num_layers,
                                  output_size=1, dropout_rate=dropout_rate).to(device)
                criterion = nn.MSELoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Entrenamiento
                num_epochs = 30
                epoch_losses = []
                for epoch in range(num_epochs):
                    epoch_loss = 0.0
                    for batch in train_loader:
                        inputs = batch[0].to(device)
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        loss = criterion(outputs, inputs[:, -1, :])
                        loss.backward()
                        optimizer.step()
                        epoch_loss += loss.item()
                    epoch_loss /= len(train_loader)
                    epoch_losses.append(epoch_loss)

                # Guardar resultados
                results.append({
                    'hidden_size': hidden_size,
                    'num_layers': num_layers,
                    'learning_rate': lr,
                    'dropout_rate': dropout_rate,
                    'loss': epoch_loss,
                    'epoch_losses': epoch_losses  # Guardar la historia de pérdidas
                })

                # Verificar si es el mejor modelo
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_params = {
                        'hidden_size': hidden_size,
                        'num_layers': num_layers,
                        'learning_rate': lr,
                        'dropout_rate': dropout_rate
                    }
                    best_model = model

# Calcular y mostrar el tiempo total de búsqueda de hiperparámetros
hyperparameter_search_time = time.time() - start_time
print(f"Tiempo de búsqueda de hiperparámetros: {hyperparameter_search_time:.2f} segundos")

# Ordenar resultados por pérdida final y seleccionar los 5 mejores
top_results = sorted(results, key=lambda x: x['loss'])[:5]

# Graficar los mejores 5 conjuntos de hiperparámetros
plt.figure(figsize=(10, 6))
for result in top_results:
    epoch_losses = result['epoch_losses']
    label = f"HS={result['hidden_size']}, NL={result['num_layers']}, LR={result['learning_rate']}, DR={result['dropout_rate']}"
    plt.plot(epoch_losses, label=label)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Comportamiento del LSTM')
# Configurar el eje X para mostrar solo números enteros
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
plt.legend()
plt.grid(True)
plt.show()

print("Mejores Hiperparámetros:", best_params)

# Obtener representaciones comprimidas del mejor modelo LSTM
with torch.no_grad():
    train_encoded = best_model(train_tensor).cpu().numpy()
    val_encoded = best_model(torch.Tensor(val_data_normalized).to(device)).cpu().numpy()


In [ ]:
# Función para calcular el Prediction Strength
def prediction_strength(train_labels, val_labels, n_clusters):
    def cluster_membership_matrix(labels, n_clusters):
        n_samples = len(labels)
        membership = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                if labels[i] == labels[j]:
                    membership[i, j] = 1
        return membership

    train_membership_matrix = cluster_membership_matrix(train_labels, n_clusters)
    val_membership_matrix = cluster_membership_matrix(val_labels, n_clusters)

    total_pairs = 0
    consistent_pairs = 0
    for i in range(min(len(train_labels), len(val_labels))):
        for j in range(i + 1, min(len(train_labels), len(val_labels))):
            if train_membership_matrix[i, j] == 1:
                total_pairs += 1
                if val_membership_matrix[i, j] == 1:
                    consistent_pairs += 1
            elif train_membership_matrix[i, j] == 0:
                total_pairs += 1
                if val_membership_matrix[i, j] == 0:
                    consistent_pairs += 1
    return consistent_pairs / total_pairs if total_pairs > 0 else 0

# Obtener representaciones comprimidas del mejor modelo LSTM
with torch.no_grad():
    train_encoded = best_model(train_tensor).cpu().numpy()
    val_encoded = best_model(torch.Tensor(val_data_normalized).to(device)).cpu().numpy()

# Prediction Strength para diferentes valores de clusters usando los mejores hiperparámetros
n_clusters_range = range(2, 21)
results_euclidean = []
results_dtw = []

for metric in ['euclidean', 'dtw']:
    print(f"\nProbando clustering con métrica: {metric.upper()}")
    for n_clusters in n_clusters_range:
        start_time = time.time()
        model = TimeSeriesKMeans(n_clusters=n_clusters, metric=metric, random_state=0)
        train_labels = model.fit_predict(train_encoded)
        val_labels = model.predict(val_encoded)
        ps_score = prediction_strength(train_labels, val_labels, n_clusters)
        execution_time = time.time() - start_time
        print(f"n_clusters = {n_clusters}, Prediction Strength = {ps_score:.2f}, Tiempo de ejecución = {execution_time:.2f} s")

        result = {
            'k': n_clusters,
            'Prediction Strength': ps_score,
            'Execution Time (s)': execution_time
        }
        if metric == 'euclidean':
            results_euclidean.append(result)
        else:
            results_dtw.append(result)

# Convertir resultados a DataFrame para graficar
df_euclidean = pd.DataFrame(results_euclidean)
df_dtw = pd.DataFrame(results_dtw)

# Graficar resultados para Euclidean
plt.figure(figsize=(10, 6))
plt.plot(df_euclidean['k'], df_euclidean['Prediction Strength'], label='Prediction Strength (Euclidean)', marker='o')
# Configurar el eje X para mostrar solo números enteros
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Prediction Strength')
plt.title('Euclidean: Prediction Strength vs Número de Clusters')
plt.legend()
plt.grid(True)
plt.show()

# Graficar resultados para DTW si hay datos
if not df_dtw.empty:
    plt.figure(figsize=(10, 6))
    plt.plot(df_dtw['k'], df_dtw['Prediction Strength'], label='Prediction Strength (DTW)', marker='o')
    # Configurar el eje X para mostrar solo números enteros
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.xlabel('Número de Clusters (K)')
    plt.ylabel('Prediction Strength')
    plt.title('DTW: Prediction Strength vs Número de Clusters')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No hay resultados para DTW.")


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.cluster import KMeans
from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import cdist_dtw
from sklearn.metrics import silhouette_score
from torch.utils.data import DataLoader, TensorDataset

# Configurar el dispositivo para usar CUDA si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando el dispositivo: {device}")

# Cargar el archivo CSV
df = pd.read_csv('generadoras_activas_corregido.csv', low_memory=False)
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Agrupar por fecha y llave, sumar 'TOTAL'
energia_por_fecha = df.groupby(['LLAVE NOMBRE', 'FECHA'])['TOTAL'].sum().reset_index()

# Pivotar los datos para tener fechas como columnas y llaves como filas
pivot_df = energia_por_fecha.pivot(index='LLAVE NOMBRE', columns='FECHA', values='TOTAL').fillna(0)
pivot_df.columns = pd.to_datetime(pivot_df.columns)
pivot_df['LLAVE NOMBRE'] = pivot_df.index  # Añadir columna con nombres de generadoras

# Usar todos los datos sin muestreo
data = pivot_df.drop(columns=['LLAVE NOMBRE']).values

# Normalización específica de series temporales
data_normalized = TimeSeriesScalerMinMax().fit_transform(data.reshape((data.shape[0], data.shape[1], 1)))

# Convertir los datos a tensores y cargar al dispositivo
data_tensor = torch.Tensor(data_normalized).to(device)
train_dataset = TensorDataset(data_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Definir el modelo Autoencoder LSTM
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, encoded_size=32, num_layers=2, dropout_rate=0.3):
        super(LSTMAutoencoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers, 
                               bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.hidden_to_encoded = nn.Linear(hidden_size * 2, encoded_size)  # Capa para codificación

        self.encoded_to_hidden = nn.Linear(encoded_size, hidden_size * 2)  # Ajuste de tamaño para el decodificador
        self.decoder = nn.LSTM(hidden_size * 2, hidden_size, num_layers=num_layers, 
                               bidirectional=True, batch_first=True)
        self.output_layer = nn.Linear(hidden_size * 2, input_size)

    def forward(self, x):
        # Encoder
        batch_size = x.size(0)
        h_0 = torch.randn(2 * self.num_layers, batch_size, self.hidden_size).to(x.device)
        c_0 = torch.randn(2 * self.num_layers, batch_size, self.hidden_size).to(x.device)
        encoder_out, _ = self.encoder(x, (h_0, c_0))
        encoder_out = self.dropout(encoder_out[:, -1, :])  # Toma la última salida del encoder
        encoded = self.hidden_to_encoded(encoder_out)  # Codificación de la serie temporal

        # Decoder
        decoder_hidden = self.encoded_to_hidden(encoded).view(2 * self.num_layers, batch_size, self.hidden_size)
        c_0_dec = torch.randn(2 * self.num_layers, batch_size, self.hidden_size).to(x.device)
        decoder_input = self.encoded_to_hidden(encoded).unsqueeze(1).repeat(1, x.size(1), 1)
        
        # Ejecutar el decodificador
        decoder_out, _ = self.decoder(decoder_input, (decoder_hidden, c_0_dec))
        out = self.output_layer(decoder_out)

        return out, encoded  # Devuelve tanto la reconstrucción como la codificación

# Configuración y entrenamiento del modelo Autoencoder LSTM
encoded_size = 32
model = LSTMAutoencoder(input_size=1, hidden_size=128, encoded_size=encoded_size, num_layers=1, dropout_rate=0.3).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
start_time = time.time()

for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch in train_loader:
        inputs = batch[0].to(device)
        optimizer.zero_grad()
        outputs, encoded = model(inputs)  # obtener salida y codificación
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

training_time = time.time() - start_time
print(f"Tiempo de entrenamiento del Autoencoder LSTM: {training_time:.2f} segundos")

# Fragmentación para evitar problemas de memoria y obtener las representaciones codificadas
with torch.no_grad():
    batch_size = 128  # Ajusta este valor según la capacidad de memoria
    encoded_data_total = []
    for i in range(0, data_tensor.size(0), batch_size):
        batch_data = data_tensor[i:i + batch_size]
        _, encoded_batch = model(batch_data)  # Solo obtenemos la codificación
        encoded_data_total.append(encoded_batch.cpu().numpy())
    encoded_data_total = np.concatenate(encoded_data_total, axis=0)

# Clustering usando KMeans (Euclidiana)
start_kmeans_time = time.time()
n_clusters = 6
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
labels_kmeans = kmeans.fit_predict(encoded_data_total)
kmeans_time = time.time() - start_kmeans_time
print(f"Tiempo de ejecución para KMeans (Euclidiana): {kmeans_time:.2f} segundos")

# Clustering usando DTW
start_dtw_time = time.time()
model_dtw = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)
labels_dtw = model_dtw.fit_predict(encoded_data_total)
dtw_time = time.time() - start_dtw_time
print(f"Tiempo de ejecución para KMeans (DTW): {dtw_time:.2f} segundos")

# Calcular la matriz de distancias DTW y el índice de silueta para DTW
dist_matrix_dtw = cdist_dtw(encoded_data_total)
np.fill_diagonal(dist_matrix_dtw, 0)
silhouette_avg_dtw = silhouette_score(dist_matrix_dtw, labels_dtw, metric="precomputed")
print(f"Índice de Silueta con DTW: {silhouette_avg_dtw:.2f}")

# Calcular el índice de silueta para KMeans (Euclidiana)
silhouette_avg_kmeans = silhouette_score(encoded_data_total, labels_kmeans)
print(f"Índice de Silueta con KMeans (Euclidiana): {silhouette_avg_kmeans:.2f}")

# Añadir etiquetas de cluster al DataFrame para KMeans y DTW
pivot_df['Cluster_KMeans'] = labels_kmeans
pivot_df['Cluster_DTW'] = labels_dtw

# Mostrar proporciones para cada tipo de generadora en cada cluster
tipo_df = df[['LLAVE NOMBRE', 'TIPO']].drop_duplicates()

def calcular_proporciones(cluster_labels, metodo):
    clusters = np.unique(cluster_labels)
    totales_cluster = {}
    porcentaje_tipo_total = {}
    total_tipos_df = tipo_df['TIPO'].value_counts().to_frame(name='Total').reset_index()
    total_tipos_df.columns = ['TIPO', 'Total']

    for cluster in clusters:
        cluster_data = pivot_df[pivot_df[f'Cluster_{metodo}'] == cluster]
        generadoras = cluster_data['LLAVE NOMBRE'].unique()
        tipos = tipo_df[tipo_df['LLAVE NOMBRE'].isin(generadoras)]
        tipo_counts = tipos['TIPO'].value_counts()
        total_cluster = tipo_counts.sum()
        totales_cluster[cluster] = total_cluster
        tipo_porcentaje_y_cantidad = tipo_counts.to_frame(name='Cantidad')
        tipo_porcentaje_y_cantidad['Porcentaje'] = (tipo_counts / total_cluster) * 100
        porcentaje_tipo_total[cluster] = tipo_porcentaje_y_cantidad

    print(f"\nProporción total de cada tipo de generadora para {metodo}:")
    print(total_tipos_df)
    for cluster, total in totales_cluster.items():
        print(f"\nCluster {cluster} - Total generadoras: {total}")
        print(porcentaje_tipo_total[cluster])

# Mostrar proporciones para KMeans y DTW
calcular_proporciones(labels_kmeans, 'KMeans')
calcular_proporciones(labels_dtw, 'DTW')

In [ ]:
# Importar las librerías necesarias
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Análisis con PCA
pca = PCA(n_components=2)
encoded_pca = pca.fit_transform(encoded_data_total)

# Visualización de PCA para KMeans
plt.figure(figsize=(10, 6))
for cluster in np.unique(labels_kmeans):
    plt.scatter(encoded_pca[labels_kmeans == cluster, 0], encoded_pca[labels_kmeans == cluster, 1], label=f'Cluster {cluster}')
plt.title("Visualización de Clústeres usando PCA")
plt.xlabel("Componente Principal 1 (Varianza Máxima)")
plt.ylabel("Componente Principal 2 (Segunda Mayor Varianza)")
plt.legend()
plt.show()

# Visualización de PCA para DTW
plt.figure(figsize=(10, 6))
for cluster in np.unique(labels_dtw):
    plt.scatter(encoded_pca[labels_dtw == cluster, 0], encoded_pca[labels_dtw == cluster, 1], label=f'Cluster {cluster}')
plt.title("Visualización de Clústeres usando PCA")
plt.xlabel("Componente Principal 1 (Varianza Máxima)")
plt.ylabel("Componente Principal 2 (Segunda Mayor Varianza)")
plt.legend()
plt.show()

# Análisis con t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=0)
encoded_tsne = tsne.fit_transform(encoded_data_total)

# Visualización de t-SNE para KMeans
plt.figure(figsize=(10, 6))
for cluster in np.unique(labels_kmeans):
    plt.scatter(encoded_tsne[labels_kmeans == cluster, 0], encoded_tsne[labels_kmeans == cluster, 1], label=f'Cluster {cluster}')
plt.title("Visualización de Clústeres usando t-SNE")
plt.xlabel("Componente t-SNE 1")
plt.ylabel("Componente t-SNE 2")
plt.legend()
plt.show()

# Visualización de t-SNE para DTW
plt.figure(figsize=(10, 6))
for cluster in np.unique(labels_dtw):
    plt.scatter(encoded_tsne[labels_dtw == cluster, 0], encoded_tsne[labels_dtw == cluster, 1], label=f'Cluster {cluster}')
plt.title("Visualización de Clústeres usando t-SNE")
plt.xlabel("Componente t-SNE 1")
plt.ylabel("Componente t-SNE 2")
plt.legend()
plt.show()